In [16]:
import pandas as pd
import numpy as np
import os

In [17]:
'''Accident Data 2019/01 - Current'''
causes = (pd.read_csv('data/accident_data.csv', on_bad_lines = 'skip',encoding = 'cp1252')
              .rename(columns={"[Date]":"Date","[Region]":"region", "[Division]":"division", "[Zone]": "zone", "[Responsibility]":"responsibility",
                               "[Causes lvl 1]":"cause1", "[Causes lvl 2]":"cause2", "[Causes lvl 3]":"cause3", 
                                  "[Responsible]":"Responsible", "[country]":"country"})
          
              .assign(Date_actual = lambda x: pd.to_datetime(x["Date"]).dt.date)
              #.loc[lambda x: x["country"] == 'Canada']
              
              .assign(Date = lambda x: x["Date"].str.slice_replace(7,10, "")) ## replacing the Date column with format year-month
              .assign(total_count = "total")
              .assign(count = 1) ## creating a column of 1s for groupby 
          
              
              .sort_values(by = "Date_actual")

          
              .drop(columns = ['[Cause ID]'])
              #.drop(columns = ["country"])
              .dropna()
              .reset_index(drop = True)
         )
#display(causes.tail())

In [18]:
'''MTAV Data 2020-01 - Current'''

mtav_causes = (pd.read_csv('data/MTAV_data.csv', on_bad_lines = 'skip',encoding = 'cp1252')
              .rename(columns={"[Date]":"Date","[country]":"country", "[violation]":"violation", "[mtav_causes]":"mtav_causes","[region]":"region", "[subregion]":"subregion",
                     "[division]":"division", "[subdivision]":"subdivision", "[responsibility]":"responsibility"})

               
              .assign(Date_actual = lambda x: pd.to_datetime(x["Date"]).dt.date)
              #.loc[lambda x: x["country"] == "Canada"]
               
              .assign(Date = lambda x: x["Date"].str.slice_replace(7,10, ""))
              .assign(Date = lambda x: x["Date"].str.slice_replace(4,5, "-"))## replacing the Date column with format year-month
              
              .assign(total = "CN Wide") ## creating a column for general sum of the entire company
              .assign(count = 1) ## creating a column of 1s for groupby
               
               
              .drop(columns = ["[counter]"])
              #.drop(columns = ["country"])
              .sort_values(by = "Date_actual", ascending = True)
              .reset_index(drop = True)
         )

#display(mtav_causes.head())


In [19]:
'''injury 2020/01 - Current'''

inj_causes = (pd.read_csv('data/injuries_data.csv', on_bad_lines = 'skip',encoding = 'cp1252')
              .rename(columns={"[Date]":"Date", "[country]":"country", "[region]":"region", "[subregion]":"division", "[division]":"zone", "[function]":"function", "[cost_center_location]":"location",
                              "[department]":"department", "[position]":"position", "[age]":"age", "[service_year]":"service_year", "[body_part]":"body_part",
                              "[phys_act]":"phys_act", "[event_circ]":"event_circ", "[inj_ill]":"inj_ill", "[sif]": "sif", "[severity]":"severity"})
              
              .assign(Date_actual = lambda x: pd.to_datetime(x["Date"]).dt.date)
              
              #.loc[lambda x: x["country"] == "Canada"]
              #.loc[lambda x: x["region"] != "INTERMODAL"]
              #.loc[lambda x: x["region"] != "FACILITY MAINT"]
              #.loc[lambda x: x["region"] != "SUPPLY MGMT"]
              #.loc[lambda x: x["region"] != "POLICE"]
              #.loc[lambda x: x["region"] != "SUPPLY CHAIN & BUS"]
              .loc[lambda x: x["region"] != "Not assigned"]


              .assign(Date = lambda x: x["Date"].str.slice_replace(7,10, "")) ## replacing the Date column with format year-month
              .assign(CN_Wide = "CN Wide") ## creating a column for general sum of the entire company
              .assign(count = 1) ## creating a column of 1s for groupby
              .assign(service_year= lambda x: x["service_year"].str.slice_replace(4,len(x), ""))
              .assign(service_year = lambda x: "Year_" + x["service_year"])
              
              
              .drop(columns = ["CN_Wide"])
              .sort_values(by = 'Date', ascending = True)
              .reset_index(drop = True)
         )

#display(inj_causes.head())


In [20]:

'''PMRC Subtest 2020/01 - Current'''

cn_org = (pd.read_csv("data/CN_org.csv")
         .rename(columns = {'Cost Center':'cc'})
         )

pmrc_sql = pd.read_csv("data/observation_pass%_sql.csv")
pmrc_sql = (pmrc_sql
            .merge(cn_org, on = 'cc', how = 'inner')
            .loc[:,['Date', 'tname', 'stname', 'cc', 'Country', 'Region', 'Division', 'Zone', 'total', 'passed', 'failed']]
            .rename(columns = {"Country":"country","Region":"region", "Division":"division", "Zone":"zone", "total":"pmrc_total"})
            .assign(Date_actual = lambda x: pd.to_datetime(x["Date"]).dt.date)
            .dropna().reset_index(drop = True)
            .drop_duplicates()
            .sort_values(by = 'Date',ascending = True)
           )



pmrc_sql = pmrc_sql.astype({
    'cc': 'int',
    'pmrc_total': 'int',
    'passed': 'int',
    'failed': 'int'
})

#pmrc_sql = pmrc_sql.loc[(pmrc_sql['tname'] == 'Safety Observation (LCR\'s)') & (pmrc_sql['region'] == 'INTERMODAL') & (pmrc_sql['stname'] == "PPE for a task (Welding/Cutting/Chemical)" )]





pmrc_sql_test = pmrc_sql.loc[(pmrc_sql['tname'] == 'Safety Observation (LCR\'s)') & (pmrc_sql['region'] == 'ENGINEERING')]
pmrc_sql_test = pmrc_sql_test.astype({
    'cc': 'int',
    'pmrc_total': 'int',
    'passed': 'int',
    'failed': 'int'
})


pmrc_sql_test = (pmrc_sql_test
                .assign(failed_perc = lambda x: x['failed'] / x['pmrc_total'] * 100)
                .loc[:,['tname', 'stname', 'region', 'division', 'failed_perc', 'pmrc_total', 'failed']]
                .groupby(['tname', 'stname', 'region', 'division'])
                .agg(
                    pmrc_total_ave=('pmrc_total', 'mean'),
                    failed_ave=('failed', 'mean'),
                    failed_perc=('failed_perc', 'mean'),  # If you want the mean failed_perc, adjust as needed
                    occurrences=('failed', 'size')  # Count occurrences for each group
                )
                .reset_index()
                .rename(columns = {'pmrc_total':"pmrc_total_ave", "failed":"failed_ave"})
                .sort_values(by = ['failed_perc'], ascending = False)
                )




In [21]:
'''Mechanical Car Defect/Fail 2021/12 - Current'''#
mech_insp = (pd.read_csv("data/mech_inspection_Q3_2024.csv")
            .rename(columns = {"date":"Date", "Sum of num_defect":"defect_count", "Sum of failed":"failed_count"})
            .assign(Date = lambda x : x["Date"].str.slice_replace(4,5, "-"))
            .assign(Date = lambda x : x["Date"].str.slice_replace(7,8, "-"))
            .assign(Date = lambda x: x["Date"].str.slice_replace(7,10, ""))
            .assign(Date_actual = lambda x: pd.to_datetime(x["Date"]).dt.date)
            .sort_values(by = "Date", ascending = False)
            .reset_index(drop =True)
             )


'''
repeated entry based on the num_defect
if num_defect / failed = 1, then no change
if num_defect / failed = 6, the entry is duplicated 6 times
NOTE: assumption that min value of Sum of num_defect is 1
'''

## filtered for defects
mech_defect = mech_insp.drop(columns = ["failed_count"])
mech_defect = mech_defect.loc[ (mech_defect["defect_cat"] != "na") | (mech_defect["car_type"] != "NA") | (mech_defect["car_type"] != "nan")]
mech_defect = mech_defect.assign(count = 1)
mech_defect_index = mech_defect.index.repeat(mech_defect['defect_count'])
mech_defect = mech_defect.loc[mech_defect_index].reset_index(drop = True)
mech_defect = mech_defect.drop(columns = ["defect_count"])


## filtered for failures
mech_failed = mech_insp.drop(columns = ["defect_count"])
mech_failed = mech_failed.assign(count = 1)
mech_failed_index = mech_failed.index.repeat(mech_failed['failed_count'])
mech_failed = mech_failed.loc[mech_failed_index].reset_index(drop = True)
mech_failed = mech_failed.drop(columns = ["failed_count", "car_type"])

date_temp = mech_insp["Date"]
date_temp = date_temp.drop_duplicates().sort_values().reset_index(drop =True)

#display(mech_defect.head())
#display(mech_failed.head())

#print(type(mech_defect.iloc[184].car_type))

In [22]:
# Get the current notebook's filename
notebook_filename = "data_prep.ipynb"  # Replace with your notebook's name

# Get the size of the notebook
size = os.path.getsize(notebook_filename)

# Convert size to kilobytes (KB) for readability
size_kb = size / 1024

print(f"The size of the notebook is: {size_kb:.2f} KB")

The size of the notebook is: 12.16 KB
